# 평가 파일

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from models.unet import UNet, compute_dice_per_class
from PIL import Image
import os

In [ ]:
img_path = "../data/SPIDER_T2_val/images/sample_T2.png"
label_path = "../data/SPIDER_T2_val/labels/sample_label.png"

img = np.array(Image.open(img_path)).astype(np.float32)
label = np.array(Image.open(label_path)).astype(np.int64)

img_norm = (img - img.min()) / (img.max() - img.min())
img_torch = torch.tensor(img_norm).unsqueeze(0).unsqueeze(0)

In [ ]:
num_classes = 14

model_A = UNet(in_channels=1, num_classes=num_classes)
model_B = UNet(in_channels=1, num_classes=num_classes)

model_A.load_state_dict(torch.load("../models/saved/model_A_synthetic.pth", map_location="cpu"))
model_B.load_state_dict(torch.load("../models/saved/model_B_real.pth", map_location="cpu"))

model_A.eval()
model_B.eval()

In [ ]:
with torch.no_grad():
    pred_A = model_A(img_torch)
    pred_B = model_B(img_torch)

pred_A = pred_A.argmax(1)[0]
pred_B = pred_B.argmax(1)[0]

In [ ]:
dice_A = compute_dice_per_class(pred_A, torch.tensor(label), num_classes=num_classes)
dice_B = compute_dice_per_class(pred_B, torch.tensor(label), num_classes=num_classes)

dice_A, dice_B

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.title("Ground Truth")
plt.imshow(label, cmap="tab20")
plt.axis("off")

plt.subplot(1,3,2)
plt.title("Prediction — Model A (Synthetic)")
plt.imshow(pred_A, cmap="tab20")
plt.axis("off")

plt.subplot(1,3,3)
plt.title("Prediction — Model B (Real)")
plt.imshow(pred_B, cmap="tab20")
plt.axis("off")

In [ ]:
classes = list(range(num_classes))
vals_A = [dice_A[c] for c in classes]
vals_B = [dice_B[c] for c in classes]

plt.figure(figsize=(12,5))
plt.bar(classes, vals_A, alpha=0.6, label="Model A Synthetic")
plt.bar(classes, vals_B, alpha=0.6, label="Model B Real")
plt.legend()
plt.title("Dice per Class Comparison (T2 Validation)")
plt.xlabel("Class ID")
plt.ylabel("Dice Score")
plt.show()
